In [27]:
import joblib
import flair
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

In [12]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
flair_model = flair.models.TextClassifier.load('en-sentiment')

def process_flair(dialogue):   # returning the flair score
    sentence = flair.data.Sentence(dialogue)
    flair_model.predict(sentence)
    label = sentence.labels[0].value
    score = sentence.labels[0].score
    if label == 'POSITIVE':
        return score
    elif label == 'NEGATIVE':
        return -score



def return_sentiment(txt):  # returning single-sentence BERT score
    encoded_input = tokenizer(txt, return_tensors='pt',padding=True,truncation=True)
    output = bert_model(**encoded_input)
    score = output[0][0].detach().numpy() 
    scores = softmax(score)
    if np.argsort(scores)[2] == 1:
        return 0
    else:
        return (np.argsort(scores)[2]-1)*scores[np.argsort(scores)[2]]
    
def tb_score(txt):
    sen = TextBlob(txt)
    return pd.Series({'tb': sen.sentiment.polarity})

def cal_vader_textblob_bert_flair(txt):
    tb_score = TextBlob(txt).sentiment.polarity
    obj = SentimentIntensityAnalyzer()
    vader_score = obj.polarity_scores(txt)['compound']
    flair_score = process_flair(txt)
    bert_score = return_sentiment(txt)
    #prob = logmodel3.predict_proba([[tb_score,vader_score,flair_score,bert_score]])[0]      
    return np.array([[vader_score, tb_score,bert_score,flair_score]])



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
from sklearn.preprocessing import FunctionTransformer

In [18]:
func_tfmr = FunctionTransformer(func=cal_vader_textblob_bert_flair)

In [20]:
# This is the wrapper
class PredictionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return self.model.predict_proba(X)

In [22]:
def fused_func(arr):
    negative = (arr[0][0]+arr[0][2])/2
    positive = (arr[0][1]+arr[0][3])/2
    #if positive >= negative:
    #    return 1
    #else:
    #    return -1
    if arr[0][1]>=arr[0][0]:
        return 1
    else:
        return -1

In [24]:
final_pipeline = joblib.load('final_pipe.pkl')

In [25]:
# Get input as a sentence from the user
input_sentence = input("Enter a sentence: ")



In [29]:
senti = final_pipeline.transform(input_sentence)

In [32]:
# Print the input sentence
print("You entered:\"",input_sentence, "\"  .....Sentiment is:", senti)

You entered:" This is a good ice cream "  .....Sentiment is: 1
